# Referencia

Para a implementação, usamos a refência [https://pt.wikipedia.org/wiki/Algoritmo_A*](https://pt.wikipedia.org/wiki/Algoritmo_A*)

# Imports

In [1]:
import heapq
from copy import deepcopy

# Methods

In [2]:
class Node:
    def __init__(
            self,
            state,
            parent=None,
            move=None,
            g=0,
            MOVE= {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)},
            GOAL_STATE= [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
        ):
        self.state = state
        self.parent = parent
        self.move = move
        self.g = g  # Custo acumulado
        self.h = self.heuristic()  # Heurística: distância de Manhattan
        self.f = self.g + self.h   # f = g + h
        self.GOAL_STATE = GOAL_STATE
        self.MOVES = MOVE

    def __lt__(self, other):
        return self.f < other.f

    def __eq__(self, other):
        return self.state == other.state

    def find_blank(self):
        """Encontra a posição do espaço vazio (0)."""
        for i in range(3):
            for j in range(3):
                if self.state[i][j] == 0:
                    return i, j

    def heuristic(self):
        """Distância de Manhattan total das peças ao objetivo."""
        distance = 0
        for i in range(3):
            for j in range(3):
                val = self.state[i][j]
                if val != 0:
                    goal_i = (val - 1) // 3
                    goal_j = (val - 1) % 3
                    distance += abs(i - goal_i) + abs(j - goal_j)
        return distance

    def get_neighbors(self):
        """Gera os vizinhos do estado atual."""
        neighbors = []
        blank_i, blank_j = self.find_blank()
        for move, (di, dj) in self.MOVES.items():
            new_i, new_j = blank_i + di, blank_j + dj
            if 0 <= new_i < 3 and 0 <= new_j < 3:
                new_state = deepcopy(self.state)
                new_state[blank_i][blank_j], new_state[new_i][new_j] = new_state[new_i][new_j], new_state[blank_i][blank_j]
                neighbors.append(Node(new_state, self, move, self.g + 1))
        return neighbors

    def path(self):
        """Reconstrói o caminho da solução."""
        node, steps = self, []
        while node:
            steps.append(node)
            node = node.parent
        return steps[::-1]

    def print_state(self):
        for row in self.state:
            print(row)

def print_frontier(frontier):
    print("Fronteira atual:")
    for node in frontier:
        print(f"  f={node.f} (g={node.g}, h={node.h}), movimento: {node.move}")
        for row in node.state:
            print("   ", row)
        print()
    print("=" * 40)

def a_star(start_state, goal_state):
    start_node = Node(start_state)
    frontier = []
    heapq.heappush(frontier, start_node)
    explored = set()
    step = 0

    while frontier:
        print(f"\n--- Etapa {step} ---")
        print_frontier(frontier)

        current = heapq.heappop(frontier)

        if current.state == goal_state:
            return current.path()

        explored.add(tuple(tuple(row) for row in current.state))

        for neighbor in current.get_neighbors():
            state_tuple = tuple(tuple(row) for row in neighbor.state)
            if state_tuple not in explored:
                heapq.heappush(frontier, neighbor)

        step += 1

    return None

# Parameters

In [3]:
# Estado final desejado
GOAL_STATE = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

# Movimentos possíveis
MOVES = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}
initial_state = [
    [1, 8, 2],
    [4, 3, 0],
    [7, 6, 5]
]

# Process

In [4]:
# ----------- EXECUÇÃO ------------
print("Executando algoritmo A*...\nEstado inicial:")
for row in initial_state:
    print(row)

solution_path = a_star(initial_state, GOAL_STATE)

if solution_path:
    print("\n🔍 Solução encontrada!")
    print("Número de movimentos:", len(solution_path) - 1)
    total_cost = solution_path[-1].g
    print("Custo total:", total_cost)

    print("\n📌 Caminho da solução:")
    for i, node in enumerate(solution_path):
        print(f"\nNível {i} | Movimento: {node.move} | g: {node.g}, h: {node.h}, f: {node.f}")
        for row in node.state:
            print(row)
else:
    print("❌ Nenhuma solução encontrada.")


Executando algoritmo A*...
Estado inicial:
[1, 8, 2]
[4, 3, 0]
[7, 6, 5]

--- Etapa 0 ---
Fronteira atual:
  f=9 (g=0, h=9), movimento: None
    [1, 8, 2]
    [4, 3, 0]
    [7, 6, 5]


--- Etapa 1 ---
Fronteira atual:
  f=9 (g=1, h=8), movimento: down
    [1, 8, 2]
    [4, 3, 5]
    [7, 6, 0]

  f=11 (g=1, h=10), movimento: up
    [1, 8, 0]
    [4, 3, 2]
    [7, 6, 5]

  f=9 (g=1, h=8), movimento: left
    [1, 8, 2]
    [4, 0, 3]
    [7, 6, 5]


--- Etapa 2 ---
Fronteira atual:
  f=9 (g=1, h=8), movimento: left
    [1, 8, 2]
    [4, 0, 3]
    [7, 6, 5]

  f=11 (g=1, h=10), movimento: up
    [1, 8, 0]
    [4, 3, 2]
    [7, 6, 5]

  f=9 (g=2, h=7), movimento: left
    [1, 8, 2]
    [4, 3, 5]
    [7, 0, 6]


--- Etapa 3 ---
Fronteira atual:
  f=9 (g=2, h=7), movimento: left
    [1, 8, 2]
    [4, 3, 5]
    [7, 0, 6]

  f=9 (g=2, h=7), movimento: down
    [1, 8, 2]
    [4, 6, 3]
    [7, 0, 5]

  f=9 (g=2, h=7), movimento: up
    [1, 0, 2]
    [4, 8, 3]
    [7, 6, 5]

  f=11 (g=1, h=10), mov

: 